In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter

In [7]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [11]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender 
from libs.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from libs.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from libs.Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

In [12]:
import optuna as op

In [7]:
# URM_train = sps.load_npz('URM_train.npz')
# URM_val = sps.load_npz('URM_val.npz')

# URM_train_val, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
# URM_train, URM_val = split_train_in_two_percentage_global_sample(URM_train_val, train_percentage = 0.85)

In [13]:
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")
URM_test = sps.load_npz("URM_test_ens.npz")

URM_train_val = sps.load_npz("URM_train_val_ens.npz")

In [14]:
print(f"Train size: {URM_train.getnnz()}, Val size: {URM_val.getnnz()}, Test size: {URM_test.getnnz()}")

Train size: 1274929, Val size: 224987, Test size: 264691


In [15]:
evaluator_val = EvaluatorHoldout(URM_val, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 527 ( 1.5%) Users that have less than 1 test interactions


In [16]:
def AP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

In [17]:
# We pass as paramether the recommender class

def evaluate_algorithm(URM_test, recommender_object, at=10):
    
    #cumulative_precision = 0.0
    #cumulative_recall = 0.0
    cumulative_AP = 0.0
    
    num_eval = 0


    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]
        
        if len(relevant_items)>0:
            
            recommended_items = recommender_object.recommend(user_id, cutoff=at)
            num_eval+=1

            #cumulative_precision += precision(recommended_items, relevant_items)
            #cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)
            
    #cumulative_precision /= num_eval
    #cumulative_recall /= num_eval
    MAP = cumulative_AP / num_eval
    
    return MAP


In [18]:
slim_model_train = SLIMElasticNetRecommender(URM_train)
#slim_model_train.fit(l1_ratio=0.11006885790633625, alpha=0.0002551115306127753, topK = 307)

slim_model_train.load_model('result_experiments/SLIM/', 'slim_307_train_ens')
# slim_model_all = SLIMElasticNetRecommender(URM_train)
# slim_model_all.load_model('result_experiments/SLIM/', 'slim_300_2_weights')

SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIM/slim_307_train_ens'
SLIMElasticNetRecommender: Loading complete


In [19]:
slim_train_res, _ = evaluator_val.evaluateRecommender(slim_model_train)

EvaluatorHoldout: Processed 34728 (100.0%) in 13.34 sec. Users per second: 2603


In [20]:
slim_train_res["MAP"]

cutoff
10    0.030621
Name: MAP, dtype: object

In [21]:
evaluate_algorithm(URM_val, slim_model_train)

0.06297588460871925

In [22]:
slim_model_train_val = SLIMElasticNetRecommender(URM_train_val)
slim_model_train_val.load_model('result_experiments/SLIM/', 'slim_307_train_val_ens')
#slim_model_train_val.fit(l1_ratio=0.11006885790633625, alpha=0.0002551115306127753, topK = 307)

SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIM/slim_307_train_val_ens'
SLIMElasticNetRecommender: Loading complete


In [23]:
slim_train_val_res, _ = evaluator_test.evaluateRecommender(slim_model_train_val)

EvaluatorHoldout: Processed 35209 (100.0%) in 14.46 sec. Users per second: 2434


In [24]:
slim_train_val_res["MAP"]

cutoff
10    0.047714
Name: MAP, dtype: object

In [25]:
evaluate_algorithm(URM_test, slim_model_train_val)

0.08366541182230076

In [56]:
rp3_beta_train = RP3betaRecommender(URM_train)
#rp3_beta_train.load_model('result_experiments/RP3beta/', 'rp3_beta_12_weights_train_ens')
#rp3_beta_train.fit(alpha=0.34989902568351894, beta=0.1817338725671425, topK=12, min_rating=0.0, implicit=False, normalize_similarity=True)
rp3_beta_train.fit(alpha=0.4176290154380183, beta=0.21520987912346945, topK=19, min_rating=0.0, implicit=False, normalize_similarity=True)

rp3_beta_train_val = RP3betaRecommender(URM_train_val)
#rp3_beta_train_val.load_model('result_experiments/RP3beta/', 'rp3_beta_12_weights_train_val_ens')
rp3_beta_train_val.fit(alpha=0.4176290154380183, beta=0.21520987912346945, topK=19, min_rating=0.0, implicit=False, normalize_similarity=True)



RP3betaRecommender: Similarity column 38121 (100.0%), 4155.69 column/sec. Elapsed time 9.17 sec
RP3betaRecommender: Similarity column 38121 (100.0%), 3808.90 column/sec. Elapsed time 10.01 sec


In [57]:
evaluate_algorithm(URM_val, rp3_beta_train), evaluate_algorithm(URM_test, rp3_beta_train_val)

(0.058588907885092474, 0.07551240567510359)

In [58]:
rp3_beta_train_res, _ = evaluator_val.evaluateRecommender(rp3_beta_train)
rp3_beta_train_val_res, _ = evaluator_test.evaluateRecommender(rp3_beta_train_val)

EvaluatorHoldout: Processed 34728 (100.0%) in 10.00 sec. Users per second: 3472
EvaluatorHoldout: Processed 35209 (100.0%) in 10.22 sec. Users per second: 3446


In [59]:
rp3_beta_train_res["MAP"], rp3_beta_train_val_res["MAP"]

(cutoff
 10    0.02844
 Name: MAP, dtype: object,
 cutoff
 10    0.042595
 Name: MAP, dtype: object)

In [30]:
# similarity='jaccard', topK=5, shrink=12, normalize=False

knn_train = ItemKNNCFRecommender(URM_train)
knn_train.load_model("result_experiments/ItemKNNCFRecommender/", "knn_4_train_ens.zip")
# knn_train.fit(similarity='tversky', topK=4, shrink=18,
#                   feature_weighting='none', tversky_alpha=0.1263621,
#                   tversky_beta=1.72181042,
#                   normalize=False)

knn_train_val = ItemKNNCFRecommender(URM_train_val)
knn_train_val.load_model("result_experiments/ItemKNNCFRecommender/", "knn_4_train_val_ens.zip")
# knn_train_val.fit(similarity='tversky', topK=4, shrink=18,
#                   feature_weighting='none', tversky_alpha=0.1263621,
#                   tversky_beta=1.72181042,
#                   normalize=False)

ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender/knn_4_train_ens.zip'
ItemKNNCFRecommender: Loading complete
ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender/knn_4_train_val_ens.zip'
ItemKNNCFRecommender: Loading complete


In [31]:
evaluate_algorithm(URM_val, knn_train), evaluate_algorithm(URM_test, knn_train_val)

(0.057754182629180306, 0.07549006255404529)

In [32]:
knn_train_res, _ = evaluator_val.evaluateRecommender(knn_train)
knn_train_val_res, _ = evaluator_test.evaluateRecommender(knn_train_val)

EvaluatorHoldout: Processed 34728 (100.0%) in 8.17 sec. Users per second: 4251
EvaluatorHoldout: Processed 35209 (100.0%) in 8.42 sec. Users per second: 4182


In [33]:
knn_train_res["MAP"], knn_train_val_res["MAP"]

(cutoff
 10    0.027949
 Name: MAP, dtype: object,
 cutoff
 10    0.042742
 Name: MAP, dtype: object)

# Optuna

In [60]:
results_dict={
    'Iteration':[],
    'SLIM_w':[],
    'RP3b_w':[],
    'ItemKNN_w':[],
    'MAP_val':[],
    'MAP_test':[],
}

In [61]:
new_best_on_test={
    'SLIM_w':0,
    'RP3b_w':0,
    'ItemKNN_w':0,
    'MAP_test':0
}

In [62]:
improving_treshold=0.85 

In [63]:
import optuna as op

best_on_val = 0.03

def objective_function_weight_hybrid(trial):
    results_dict['Iteration'].append(trial.number)
    n = 3
    x = []
    for i in range(n):
        x.append(- np.log(trial.suggest_float(f"x_{i}", 0, 1)))
    weights = []
    for i in range(n):
        weights.append(x[i] / sum(x))
        if i==0:
            key='SLIM_w'
        elif i==1:
            key='RP3b_w'
        else:
            key='ItemKNN_w'
        results_dict[key].append(weights[i])

    for i in range(n):
        trial.set_user_attr(f"weights_{i}",weights[i])
        
    #PRINT WEIGHTS CHOSEN ###################################################
    print("__________Iteration " + str(trial.number) + "______________")
    print("SLIM weight: "+str(weights[0]))
    print("RP3b weight: "+str(weights[1]))
    print("ItemKNN weight: "+str(weights[2]))##
    
    #BUILD HYBRID ###########################################################
    new_similarity_train = weights[0] * slim_model_train.W_sparse + weights[1] * rp3_beta_train.W_sparse + weights[2] * knn_train.W_sparse
    recommender_object_train = ItemKNNCustomSimilarityRecommender(URM_train)
    recommender_object_train.fit(new_similarity_train)
    res_on_val, _ = evaluator_val.evaluateRecommender(recommender_object_train)
    results_dict['MAP_val'].append(res_on_val['MAP'][10])

    #print("VAL", res_on_val['MAP'][10], best_on_val * improving_treshold)
    if res_on_val['MAP'][10] >= best_on_val * improving_treshold:
        new_similarity_train_val = weights[0] * slim_model_train_val.W_sparse + weights[1] * rp3_beta_train_val.W_sparse + weights[2] * knn_train_val.W_sparse
        recommender_object_train_val = ItemKNNCustomSimilarityRecommender(URM_train_val)
        recommender_object_train_val.fit(new_similarity_train_val)
        res_on_test, _ = evaluator_test.evaluateRecommender(recommender_object_train_val)
        results_dict['MAP_test'].append(res_on_test["MAP"][10])

        #print("Test", res_on_test["MAP"][10], new_best_on_test['MAP_test'])
        if res_on_test["MAP"][10] > new_best_on_test['MAP_test']:
            new_best_on_test['SLIM_w']=weights[0]
            new_best_on_test['RP3b_w']=weights[1]
            new_best_on_test['ItemKNN_w']=weights[2]
            new_best_on_test['MAP_test']=res_on_test["MAP"][10]
            print("New best on test found! Score is "+ str(res_on_test["MAP"][10]))
            print(f"weights: SLIM_w: {weights[0]}, RP3b_w: {weights[1]}, ItemKNN_w: {weights[2]}")
            
        
        
    else:
         results_dict['MAP_test'].append(0.0)
    return res_on_val['MAP'][10]

In [64]:
study = op.create_study(direction="maximize")
study.optimize(objective_function_weight_hybrid,
                      callbacks=[],
                      n_trials = 500)

[I 2024-12-18 00:53:18,473] A new study created in memory with name: no-name-9c6f5412-3f52-4fe7-b001-f6b6709bd811


__________Iteration 0______________
SLIM weight: 0.1555905222316807
RP3b weight: 0.4603506048669601
ItemKNN weight: 0.38405887290135915
EvaluatorHoldout: Processed 34728 (100.0%) in 15.33 sec. Users per second: 2265
EvaluatorHoldout: Processed 35209 (100.0%) in 16.60 sec. Users per second: 2120


[I 2024-12-18 00:53:50,624] Trial 0 finished with value: 0.030289843198662226 and parameters: {'x_0': 0.46359243423077034, 'x_1': 0.10284590795296422, 'x_2': 0.14993140055314513}. Best is trial 0 with value: 0.030289843198662226.


New best on test found! Score is 0.04628595141844424
weights: SLIM_w: 0.1555905222316807, RP3b_w: 0.4603506048669601, ItemKNN_w: 0.38405887290135915
__________Iteration 1______________
SLIM weight: 0.13670679085741672
RP3b weight: 0.5253840472394294
ItemKNN weight: 0.33790916190315384
EvaluatorHoldout: Processed 34728 (100.0%) in 14.71 sec. Users per second: 2360
EvaluatorHoldout: Processed 35209 (100.0%) in 15.06 sec. Users per second: 2338


[I 2024-12-18 00:54:20,547] Trial 1 finished with value: 0.03035995953130447 and parameters: {'x_0': 0.6943600406456691, 'x_1': 0.2461427659491442, 'x_2': 0.4059123084250371}. Best is trial 1 with value: 0.03035995953130447.


...

[I 2024-12-18 04:46:43,321] Trial 497 finished with value: 0.03107241926371922 and parameters: {'x_0': 0.09037765443603266, 'x_1': 0.40290779545452216, 'x_2': 0.7871147639312055}. Best is trial 203 with value: 0.031075370772588123.


__________Iteration 498______________
SLIM weight: 0.6568606713364563
RP3b weight: 0.2654444004017231
ItemKNN weight: 0.07769492826182053
EvaluatorHoldout: Processed 34728 (100.0%) in 12.87 sec. Users per second: 2698
EvaluatorHoldout: Processed 35209 (100.0%) in 13.96 sec. Users per second: 2523


[I 2024-12-18 04:47:10,241] Trial 498 finished with value: 0.031069103244076277 and parameters: {'x_0': 0.0817924622809566, 'x_1': 0.3635932778276494, 'x_2': 0.7436923773495275}. Best is trial 203 with value: 0.031075370772588123.


__________Iteration 499______________
SLIM weight: 0.721499899182245
RP3b weight: 0.2171641142458087
ItemKNN weight: 0.06133598657194633
EvaluatorHoldout: Processed 34728 (100.0%) in 12.83 sec. Users per second: 2707
EvaluatorHoldout: Processed 35209 (100.0%) in 13.94 sec. Users per second: 2526


[I 2024-12-18 04:47:37,100] Trial 499 finished with value: 0.031023643380026226 and parameters: {'x_0': 0.04464387935663588, 'x_1': 0.3922763311626962, 'x_2': 0.767739464813105}. Best is trial 203 with value: 0.031075370772588123.


In [ ]:
similarity_best = 0.862708291473966 * slim_model_train_val.W_sparse + 0.10238146945917358 * rp3_beta_train_val.W_sparse + 0.034910239066860506 * knn_train_val.W_sparse
recommender_best = ItemKNNCustomSimilarityRecommender(URM_train_val)
recommender_best.fit(similarity_best)

In [67]:
results_df=pd.DataFrame(results_dict)
results_df.to_csv("result_experiments/ensemble_SLIM_RP3_KNNCFR/results_prof_eval_18_12_24.csv",sep=',',index=False)

In [ ]:
res_best, _ = evaluator_test.evaluateRecommender(recommender_best)

In [ ]:
res_best

In [35]:
results_best[results_best["MAP_test"]==results_best["MAP_test"].max()]

,Iteration,SLIM_w,RP3b_w,ItemKNN_w,MAP_val,MAP_test
148,148,0.867298,0.105076,0.027626,0.030877,0.047923


In [66]:
new_best_on_test

{'SLIM_w': 0.8011551243595696,
 'RP3b_w': 0.176441040480307,
 'ItemKNN_w': 0.022403835160123328,
 'MAP_test': 0.0480096505357787}

In [88]:
# slim_model_train.save_model("result_experiments/SLIM/", "slim_307_train_ens")
# slim_model_train_val.save_model("result_experiments/SLIM/", "slim_307_train_val_ens")

In [89]:
# knn_train.save_model("result_experiments/ItemKNNCFRecommender/", "knn_4_train_ens")
# knn_train_val.save_model("result_experiments/ItemKNNCFRecommender/", "knn_4_train_val_ens")

In [90]:
# rp3_beta_train.save_model("result_experiments/RP3beta/", "rp3_beta_12_weights_train")
# rp3_beta_train_val.save_model("result_experiments/RP3beta/", "rp3_beta_12_weights_train_val")

In [57]:
# results_df=pd.DataFrame(results_dict)
# results_df.to_csv("result_experiments/ensemble_SLIM_RP3_KNNCFR/results_prof_eval_11_12_24.csv",sep=',',index=False)

In [63]:
# sps.save_npz('URM_train_ens.npz', URM_train)
# sps.save_npz('URM_train_val_ens.npz', URM_train_val)
# sps.save_npz('URM_test_ens.npz', URM_test)

In [70]:
print(new_best_on_test['SLIM_w'])
print(new_best_on_test['RP3b_w'])
print(new_best_on_test['ItemKNN_w'])

0.8011551243595696
0.176441040480307
0.022403835160123328


In [70]:
results_df.sort_values('MAP_val', ascending=False)

,Iteration,SLIM_w,RP3b_w,ItemKNN_w,MAP_val,MAP_test
267,267,0.000138,0.999847,0.000015,0.128963,0.0
257,257,0.000048,0.999820,0.000133,0.128963,0.0
297,297,0.000019,0.999837,0.000144,0.128961,0.0
260,260,0.000519,0.999391,0.000089,0.128960,0.0
253,253,0.000198,0.999592,0.000210,0.128958,0.0
...,...,...,...,...,...,...
9,9,0.415903,0.133992,0.450105,0.089874,0.0
1,1,0.752678,0.061503,0.185818,0.089587,0.0
237,237,0.052045,0.195269,0.752685,0.086758,0.0
30,30,0.013971,0.168956,0.817073,0.084429,0.0


In [69]:
def write_submission_lib_model(trained_model, filename: str = "submission.csv") -> None:
	"""Builds the submission file from a trained recommender model. The file is saved in a CSV format.

	:param trained_model: A fitted recommender model
	:type trained_model: RecommenderModel
	:param filename: The filename of the submission for this particular recommender model
	:type filename: str
	"""
	target_users_test = pd.read_csv("../data/data_target_users_test.csv",).to_numpy().ravel()

	recommendations = np.array([
		trained_model.recommend(user_id, cutoff=10) for user_id in target_users_test
	])

	if not os.path.exists("../submissions"):
		os.makedirs("../submissions")
	with open(f"../submissions/{filename}", "w") as f:
		f.write("user_id,item_list\n")
		for user_id, recs in zip(target_users_test, recommendations):
			f.write(f"{user_id},{' '.join(map(str, recs))}\n")

In [71]:
SLIM_all = SLIMElasticNetRecommender(URM_all)
# SLIM_all.fit(l1_ratio=0.11006885790633625, alpha=0.0002551115306127753, topK = 307)
SLIM_all.load_model("result_experiments/SLIM/", "slim_307_weights_all")


SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIM/slim_307_weights_all'
SLIMElasticNetRecommender: Loading complete


In [72]:
RP3_beta_all = RP3betaRecommender(URM_all)
RP3_beta_all.fit(alpha=0.4176290154380183, beta=0.21520987912346945, topK=19, min_rating=0.0, implicit=False, normalize_similarity=True)

#RP3_beta_all.load_model("result_experiments/RP3beta/", "rp3_beta_12_weights_all")


RP3betaRecommender: Similarity column 38121 (100.0%), 3554.66 column/sec. Elapsed time 10.72 sec


In [73]:
KNN_all = ItemKNNCFRecommender(URM_all)
# KNN_all.fit(similarity='tversky', topK=4, shrink=18,
#                   feature_weighting='none', tversky_alpha=0.1263621,
#                   tversky_beta=1.72181042,
#                   normalize=False)

KNN_all.load_model("result_experiments/ItemKNNCFRecommender/", "knn_4_weights_all")

ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender/knn_4_weights_all'
ItemKNNCFRecommender: Loading complete


In [74]:
# iter 239 0.04801274656744886
# 0.7380796311636464 * SLIM_all.W_sparse + 0.21602197286950797 * RP3_beta_all.W_sparse + 0.04589839596684567 * KNN_all.W_sparse

similarity_all = 0.8011551243595696 * SLIM_all.W_sparse + 0.176441040480307 * RP3_beta_all.W_sparse + 0.022403835160123328 * KNN_all.W_sparse
recommender_best = ItemKNNCustomSimilarityRecommender(URM_all)
recommender_best.fit(similarity_all)

In [75]:
recommender_best_res, _ = evaluator_test.evaluateRecommender(recommender_best)

EvaluatorHoldout: Processed 35209 (100.0%) in 15.03 sec. Users per second: 2343


In [76]:
recommender_best_res

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.985253,0.0,0.985253,0.25946,13.573524,0.999895,0.424386,0.919259,1.269521,0.196495


In [77]:
write_submission_lib_model(recommender_best, "ensemble_SLIM_RP3beta_KNN_9.csv")

In [72]:
recommender_best.save_model("result_experiments/ensemble_SLIM_RP3_KNNCFR/", "ensemble_weights_11_12_24")

ItemKNNCustomSimilarityRecommender: Saving model in file 'result_experiments/ensemble_SLIM_RP3_KNNCFR/ensemble_weights_11_12_24'
ItemKNNCustomSimilarityRecommender: Saving complete


In [73]:
SLIM_all.save_model("result_experiments/SLIM/", "slim_307_weights_all")

SLIMElasticNetRecommender: Saving model in file 'result_experiments/SLIM/slim_307_weights_all'
SLIMElasticNetRecommender: Saving complete


In [74]:
RP3_beta_all.save_model("result_experiments/RP3beta/", "rp3_beta_12_weights_all")

RP3betaRecommender: Saving model in file 'result_experiments/RP3beta/rp3_beta_12_weights_all'
RP3betaRecommender: Saving complete


In [75]:
KNN_all.save_model("result_experiments/ItemKNNCFRecommender/", "knn_4_weights_all")

ItemKNNCFRecommender: Saving model in file 'result_experiments/ItemKNNCFRecommender/knn_4_weights_all'
ItemKNNCFRecommender: Saving complete
